# Opérations de type vivit

L'idée est travailler sur des données de type vidéo et de comprendre comment les manipuler selon les differents modeles de vivit (model1, model2, model3, model4)

Déja, on fait les install et les imports

In [ ]:
!pip install torchviz

In [ ]:
import numpy as np
import torch
from torchviz import make_dot

import einops

device = 'cuda' if torch.cuda.is_available() else 'cpu'


### Manipulations spatio temporelles sur des videos

A partir d'ici, on a compris que si on rajoute une dimension Batch, les opérations se feront sans difficultés sur chacun des items du batch par brodcasting.

On ne travaillera donc plus, pour ces démos, avec des données batchées.

#### Préparation d'une vidéo exemple

Pour simplifier, on n'extrait plus de patchs, on va directement créer des patch vectorisés avec des tailles réduites qui nous arrangent

- un patch a un embedding de taille 4 (eventuellement 2x1x2)
- pour un indice temporel, on a 3 patchs spatiaux differents (eventuellement 3x1)
- on a 2 indices temporels.

Nos données ont une shape : $(npt,nph \times npw, embed)$ = [2,3,4]

In [ ]:
# 3 trucs inutiles dans la pratique ca donne juste la longueur de l'embedding
hp = 1
wp = 2
c = 2
embed_dim = hp*wp*c

# 2 trucs inutiles, ca sert juste a donner le nombre de patchs spatiaux
nph = 3
npw = 1

nps = nph*npw

# nombre de patchs temporels
npt = 2

# on cree un premier patch
p1 = torch.tensor([i+1 for i in range(embed_dim)])

print("p1\n",p1)

# on crée les différents patchs spatiaux pour p1
list_ps1 = [p1*(10**(i)) for i in range(nps)]

ps1 = torch.stack(list_ps1)

patches = einops.rearrange([ps1, -ps1],"npt nps embed_dim-> npt nps embed_dim")

patches = einops.rearrange(patches,"npt nps embed_dim-> npt nps embed_dim")

print ("\npatches\n", patches)
print ("\npatches\n", patches.shape)


p1
 tensor([1, 2, 3, 4])

patches
 tensor([[[   1,    2,    3,    4],
         [  10,   20,   30,   40],
         [ 100,  200,  300,  400]],

        [[  -1,   -2,   -3,   -4],
         [ -10,  -20,  -30,  -40],
         [-100, -200, -300, -400]]])

patches
 torch.Size([2, 3, 4])


### Encodage et traitement spatial, ou temporel, ou les deux


Dans ce qui précède, on a pris nos données, encodé chaque patch (tubelet) en vecteurs, et les différents tubelets sont encodés en vecteurs. J'ai donc une matrice pour la vidéo.

pour ne pas se mélanger, voici la shape de nos données : $[npt ,(nph \times npw), (embed)]$

Nos traitements vont consister à appliquer une attention bidon à nos données.

une **attention** va consister à ajouter à chaque token un vecteur dépendant des données considérées. par exemple, **le max**.



In [ ]:
def attention(x):
  maxi,i_maxi = torch.max(x,dim=-2)
  return torch.unsqueeze(maxi,-2)


#### Travail spatio temporel

Pour un travail "spatio-temporel", au sens du **model 1 des vivit** : les transformers calculent l'attention entre tous les patchs (spatiaux et temporels)


**il suffit de fusionner les dimensions correspondant à nt et ns**.

- traitement : vu nos données, on va ajouter à chaque token le vecteur $[100,200,300,400]$.

In [ ]:

print("\n=====preparation pour travail spatiotemporel=====\n")
print("\n patches \n",patches)
print("\n patches.shape \n",patches.shape)

p_spatio_temp = einops.rearrange(patches,"npt nps embed -> (npt nps) embed")
print("\np_spatio_temp\n",p_spatio_temp)
print("\np_spatio_temp.shape\n",p_spatio_temp.shape)
n_patch,embed_dim = p_spatio_temp.shape

#att_spatio_temp = torch.eye(embed_dim,embed_dim)*1.1
#print (att_spatio_temp)
#print ("att_spatio_temp.shape",att_spatio_temp.shape)

maxi = attention(p_spatio_temp)
print("\n maxi.shape\n",maxi.shape)
print("\n maxi\n",maxi)
res = p_spatio_temp + maxi
print("\n res\n",res)
print("\n res.shape",res.shape)
print(" parfait ")




=====preparation pour travail spatiotemporel=====


 patches 
 tensor([[[   1,    2,    3,    4],
         [  10,   20,   30,   40],
         [ 100,  200,  300,  400]],

        [[  -1,   -2,   -3,   -4],
         [ -10,  -20,  -30,  -40],
         [-100, -200, -300, -400]]])

 patches.shape 
 torch.Size([2, 3, 4])

p_spatio_temp
 tensor([[   1,    2,    3,    4],
        [  10,   20,   30,   40],
        [ 100,  200,  300,  400],
        [  -1,   -2,   -3,   -4],
        [ -10,  -20,  -30,  -40],
        [-100, -200, -300, -400]])

p_spatio_temp.shape
 torch.Size([6, 4])

 maxi.shape
 torch.Size([1, 4])

 maxi
 tensor([[100, 200, 300, 400]])

 res
 tensor([[101, 202, 303, 404],
        [110, 220, 330, 440],
        [200, 400, 600, 800],
        [ 99, 198, 297, 396],
        [ 90, 180, 270, 360],
        [  0,   0,   0,   0]])

 res.shape torch.Size([6, 4])
 parfait 




#### Travail spatial puis temporel : model 2 vivit (factorized encoder)

Dans le cas du **modele 2 des vivit** :
- on a un transformer spatial (avec eventuellement plein de couches)
- puis on selectionne un représentant de chaque indice temporel
- on a un transformer temporel (avec eventuellement plein de couches).


**preparation pour traitement spatial**

- On traite chaque indice temporel indépendamment. Cet indice temporel contient toutes les dimensions spatiales de ces moments.

c'est l'equivalent d'un batch de tous les indices temporels.

C'est déja la forme de nos données

- le coeur de mon attention est calculé sur une liste de *ns* vecteurs $n_{seq} = (nph \times npw)$ (n_seq = 2x2 = 4)
- chaque vecteur est de taille *embed_dim* : $embed_{dim} = (fp \times c \times hp \times wp)$ (embed_dim = 8x2x3x2 = 24

on a $npt$ indices temporels différents => je calcule ca par batch.
Le resultat est de taille $npt \times nseq \times embed_{dim}$ : 4x4x24

ces inputs sont injectés alors dans l'encodeur spatial

**sélection d'un représentant d'un indice temporel**

Dans ce modele, après avoir modifié tous les patchs en fonction des patchs de meme indice temporel, on selectionne **un seul patch** de chaque indice temporel.

Ce peut être :
- le premier (**cls**)
- un global average.

ici, on va prendre le premier, c'est plus facile à visualiser.

le résultat de cette sélection est une liste de $npt$ patchs, chacun de taille $embed_dim$, soit 4 patchs de taille 24

**traitement temporel**

on injecte ces inputs dans le transformeur temporel



In [ ]:


print("\n=====preparation pour travail spatial=====\n")

p_spatial = patches
print(p_spatial)
print("\np_spatial.shape\n",p_spatial.shape)
print("PARFAIT")

# on applique la transfo spatiale
print("\t ------------ transfo spatiale -------------")

maxi = attention(p_spatial)
print("\nmaxi\n",maxi)
print("\nmaxi.shape\n",maxi.shape)
res_spatial = p_spatial + maxi

print("\nres_spatial\n",res_spatial)
print("\nres.shape",res_spatial.shape)
print("PARFAIT")

# on selectionne le premier token de chaque indice temporel
p_temp = res_spatial[:,0]

print("\t ------------ selection d'un token spatial par indice temporel  -------------")
print("p_temp.shape",p_temp.shape)
print("\np_temp\n",p_temp.int())
print("PARFAIT")

# on applique la transfo temporelle
print("\t ------------ transfo temporelle -------------")
maxi = attention(p_temp)
print("\nmaxi\n",maxi)
print("\nmaxi.shape\n",maxi.shape)
res_temp = p_temp + maxi


print("res_temp.shape",res_temp.shape)
print("\nres_temp\n",res_temp.int())
print("PARFAIT")




=====preparation pour travail spatial=====

tensor([[[   1,    2,    3,    4],
         [  10,   20,   30,   40],
         [ 100,  200,  300,  400]],

        [[  -1,   -2,   -3,   -4],
         [ -10,  -20,  -30,  -40],
         [-100, -200, -300, -400]]])

p_spatial.shape
 torch.Size([2, 3, 4])
PARFAIT
	 ------------ transfo spatiale -------------

maxi
 tensor([[[100, 200, 300, 400]],

        [[ -1,  -2,  -3,  -4]]])

maxi.shape
 torch.Size([2, 1, 4])

res_spatial
 tensor([[[ 101,  202,  303,  404],
         [ 110,  220,  330,  440],
         [ 200,  400,  600,  800]],

        [[  -2,   -4,   -6,   -8],
         [ -11,  -22,  -33,  -44],
         [-101, -202, -303, -404]]])

res.shape torch.Size([2, 3, 4])
PARFAIT
	 ------------ selection d'un token spatial par indice temporel  -------------
p_temp.shape torch.Size([2, 4])

p_temp
 tensor([[101, 202, 303, 404],
        [ -2,  -4,  -6,  -8]], dtype=torch.int32)
PARFAIT
	 ------------ transfo temporelle -------------

maxi
 tensor(

#### block = encodeur spatial puis encodeur temporel : model 3 **factorized self attention**

Dans le model 3 (factorized self attention), un block d'encoder est la succession d'un encodeur spatial et d'un encodeur temporel.

pour l'encodeur spatial, c'est la meme chose que celui d'avant.

pour l'encodeur temporel, on calcule, pour chaque token spatial, l'attention entre tous les token de meme position, mais d'indice temporel différent.

On va deja coder ca



In [ ]:

print("\n =====preparation pour travail temporel====\n")
# shape avant : [ni_patch * nj_patch, frames,C,Hp,Wp]
p_temp = einops.rearrange(patches,"npt nps embed -> nps npt embed ")
print(p_temp)
print("\np_temp.shape\n",p_temp.shape)

# on applique la transfo temporelle
print("\t ------------ transfo temporelle -------------")
maxi = attention(p_temp)
print("\nmaxi\n",maxi)
print("\nmaxi.shape\n",maxi.shape)
res_temp = p_temp + maxi

print("res_temp.shape",res_temp.shape)
print("\nres_temp\n",res_temp.int())
print("PARFAIT")





 =====preparation pour travail temporel====

tensor([[[   1,    2,    3,    4],
         [  -1,   -2,   -3,   -4]],

        [[  10,   20,   30,   40],
         [ -10,  -20,  -30,  -40]],

        [[ 100,  200,  300,  400],
         [-100, -200, -300, -400]]])

p_temp.shape
 torch.Size([3, 2, 4])
	 ------------ transfo temporelle -------------

maxi
 tensor([[[  1,   2,   3,   4]],

        [[ 10,  20,  30,  40]],

        [[100, 200, 300, 400]]])

maxi.shape
 torch.Size([3, 1, 4])
res_temp.shape torch.Size([3, 2, 4])

res_temp
 tensor([[[  2,   4,   6,   8],
         [  0,   0,   0,   0]],

        [[ 20,  40,  60,  80],
         [  0,   0,   0,   0]],

        [[200, 400, 600, 800],
         [  0,   0,   0,   0]]], dtype=torch.int32)
PARFAIT


**realisation d'un bloc encoder model3**

Vu que ce qui précède fonctionne, on peut maintenant chainer les 2 :
1. encodeur spatial
2. permutations pour encodeur spatial
3. encodeur temporel
4. permutation pour passer à la suite.


In [ ]:
print("\n=====preparation pour travail spatial=====\n")

p_spatial = patches
print(p_spatial)
print("\np_spatial.shape\n",p_spatial.shape)

# on applique la transfo spatiale
print("\t ------------ transfo spatiale -------------")

maxi = attention(p_spatial)
print("\nmaxi\n",maxi)
print("\nmaxi.shape\n",maxi.shape)
res_spatial = p_spatial + maxi

print("\nres_spatial\n",res_spatial)
print("\nres.shape",res_spatial.shape)

print("\n =====preparation pour travail temporel====\n")
# shape avant : [ni_patch * nj_patch, frames,C,Hp,Wp]
p_temp = einops.rearrange(res_spatial,"npt nps embed -> nps npt embed ")
print("\ninput de l'attention temporelle\n",p_temp)
print("\np_temp.shape\n",p_temp.shape)

# on applique la transfo temporelle
print("\t ------------ transfo temporelle -------------")
maxi = attention(p_temp)
print("\nmaxi\n",maxi)
print("\nmaxi.shape\n",maxi.shape)
res_temp = p_temp + maxi

print("res_temp.shape",res_temp.shape)
print("\nres_temp\n",res_temp.int())

print("\n =====preparation pour poursuite====\n")
out = einops.rearrange(res_temp,"nps npt embed -> npt nps embed ")
print("\nout\n",out)


=====preparation pour travail spatial=====

tensor([[[   1,    2,    3,    4],
         [  10,   20,   30,   40],
         [ 100,  200,  300,  400]],

        [[  -1,   -2,   -3,   -4],
         [ -10,  -20,  -30,  -40],
         [-100, -200, -300, -400]]])

p_spatial.shape
 torch.Size([2, 3, 4])
	 ------------ transfo spatiale -------------

maxi
 tensor([[[100, 200, 300, 400]],

        [[ -1,  -2,  -3,  -4]]])

maxi.shape
 torch.Size([2, 1, 4])

res_spatial
 tensor([[[ 101,  202,  303,  404],
         [ 110,  220,  330,  440],
         [ 200,  400,  600,  800]],

        [[  -2,   -4,   -6,   -8],
         [ -11,  -22,  -33,  -44],
         [-101, -202, -303, -404]]])

res.shape torch.Size([2, 3, 4])

 =====preparation pour travail temporel====


input de l'attention temporelle
 tensor([[[ 101,  202,  303,  404],
         [  -2,   -4,   -6,   -8]],

        [[ 110,  220,  330,  440],
         [ -11,  -22,  -33,  -44]],

        [[ 200,  400,  600,  800],
         [-101, -202, -303,

#### Model 4: Factorised dot-product attention

Bon, là, faut relire
